# Running LLAMA3 on Intel AI PCs

## Introduction

This notebook demonstrates how to install Ollama on Windows with Intel GPUs. It applies to Intel Core Ultra and Core 11 - 14 gen integrated GPUs (iGPUs), as well as Intel Arc Series GPU.

## What is an AIPC

What is an AI PC you ask?

Here is an [explanation](https://www.intel.com/content/www/us/en/newsroom/news/what-is-an-ai-pc.htm#gs.a55so1):

”An AI PC has a CPU, a GPU and an NPU, each with specific AI acceleration capabilities. An NPU, or neural processing unit, is a specialized accelerator that handles artificial intelligence (AI) and machine learning (ML) tasks right on your PC instead of sending data to be processed in the cloud. The GPU and CPU can also process these workloads, but the NPU is especially good at low-power AI calculations. The AI PC represents a fundamental shift in how our computers operate. It is not a solution for a problem that didn’t exist before. Instead, it promises to be a huge improvement for everyday PC usages.”

## Install Prerequisites

### Step 1: System Preparation

To set up your AIPC for running with Intel iGPUs, follow these essential steps:

1. Update Intel GPU Drivers: Ensure your system has the latest Intel GPU drivers, which are crucial for optimal performance and compatibility. You can download these directly from Intel's [official website](https://www.intel.com/content/www/us/en/download/785597/intel-arc-iris-xe-graphics-windows.html) . Once you have installed the official drivers, you could also install Intel ARC Control to monitor the gpu:

   <img src="Assets/gpu_arc_control.png">


2. Install Visual Studio 2022 Community edition with C++: Visual Studio 2022, along with the “Desktop Development with C++” workload, is required. This prepares your environment for C++ based extensions used by the intel SYCL backend that powers accelerated Ollama. You can download VS 2022 Community edition from the official site, [here](https://visualstudio.microsoft.com/downloads/).

3. Install conda-forge: conda-forge will manage your Python environments and dependencies efficiently, providing a clean, minimal base for your Python setup. Visit conda-forge's [installation site](https://conda-forge.org/download/) to install for windows.

4. Install Intel oneAPI Base Toolkit: The oneAPI Base Toolkit (specifically Intel’ SYCL runtime, MKL and OneDNN) is essential for leveraging the performance enhancements offered by Intel's libraries and for ensuring that Ollama can fully utilize the GPU capabilities. By following these steps, your AI PC will be primed for running Ollama leveraging Intel iGPUs.

```
conda create -n llm-ollama python=3.11 -y
conda activate llm-ollama
conda install libuv -y
pip install dpcpp-cpp-rt==2024.0.2 mkl-dpcpp==2024.0.0 onednn==2024.0.0

```

   

## Step 2: Install Ollama with Intel GPU support

* Now that we have set up the environment, Intel GPU drivers, and runtime libraries, we can configure ollama to leverage the on-chip GPU.
* Open miniforge prompt and run the below commands. We Install IPEX-LLM for llama.cpp and to use llama.cpp with IPEX-LLM, first ensure that ipex-llm[cpp] is installed.

### With the ollama environment active, use pip to install required libraries for GPU. 
```
conda activate llm-ollama
pip install --pre --upgrade ipex-llm[cpp]
```

<img src="Assets/llm14.png">

* Create a folder ollama and navigate to the folder

  ```
  mkdir ollama
  cd ollama
  ```
<img src="Assets/llm15.png">

* Open another miniforge prompt in administrator privilege mode and run the below command.
    
* Navigate to the above "ollama" folder that you created and run the below commands
  
    ```
    conda activate llm-ollama
    init-ollama.bat  # if init-ollama.bat is not available in your environment, restart your terminal

    ```
    <img src="Assets/llm17.png">

* Open another Miniforge prompt, navigate to the ollama folder where we created the symbolic links above and run the below command

  ```
  ollama serve

  ```
* ollama is now running in the backend and we should see as below

  <img src="Assets/llm18.png">
    

## Run llama3 using Ollama on AI PC

Now that we have installed Ollama, let’s see how to run llama 3 on your AI PC!
Pull the Llama 3 8b from ollama repo:

```
ollama pull llama3

```
<img src="Assets/llm20.png">

*  Now, let’s create a custom llama 3 model and also configure all layers to be offloaded to the GPU.
*  The main settings in the configuration file include num_gpu, which is set to 999 to ensure all layers utilize the GPU. We also configured the context length to 8192, the maximum supported by Llama 3.
*  Additionally, we  customized the system prompt to add a more playful touch to the assistant (Pika :)). Here is a sample [Model file](Modelfile/Modelfile.llama3).

<img src="Assets/model_file.png">


* Now that we have created a custom Modelfile, let’s create a custom model:

```
ollama create llama3-gpu -f Modelfile/Modelfile.llama3

```

* Let’s see if the model got created. The new model is ready to be run!.

  <img src="Assets/llm21.png">

* Finally, now let’s run the model.
```
ollama run llama3-gpu

```

* As you can see above llama 3 is running on iGPU on the AI PC.

<img src="Assets/llm22.png">
  

## Example code to run the models using streamlit on AI PC

In [ ]:
%%writefile src/st_ollama.py
import ollama
import streamlit as st

st.title("Let's Chat....🐼")

# Load ollama models

model_list = [model["name"] for model in ollama.list()["models"]]
model = st.selectbox("Choose a model from the list", model_list)

if chat_input := st.chat_input("Hi, How are you?"):
    with st.spinner("Running....🐎"):
        with st.chat_message("user"):
            st.markdown(chat_input)

        def generate_response(user_input):
            response = ollama.chat(model=model, messages=[
            {
                'role': 'user',
                'content': chat_input,
            },
            ],
            stream=True,
            )    
            for res in response:
                yield res["message"]["content"]            
        st.write_stream(generate_response(chat_input))
        del model

In [ ]:
! streamlit run src/st_ollama.py

### Streamlit output runnling llama3

Below is the screesnhot of llama3 is running on iGPU on the AI PC.

<img src="Assets/ollama.png">


* Reference: https://ipex-llm.readthedocs.io/en/latest/doc/LLM/Quickstart/ollama_quickstart.html